# VacationPy 

### This part will use data skills to plan future vacations
- The main tasks will be to use the Geoapify API and the geoViews Python library and employ your Python skills to create map visualizations.

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tongchuan,35.0806,109.0897,9.52,26,0,3.95,CN,1701832175
1,1,paracuru,-3.4100,-39.0306,28.39,82,28,5.94,BR,1701832175
2,2,blackmans bay,-43.0167,147.3167,21.40,36,6,1.36,AU,1701832175
3,3,ilulissat,69.2167,-51.1000,-10.99,52,6,3.60,GL,1701832163
4,4,great falls,47.5002,-111.3008,14.67,51,20,7.72,US,1701832175


## Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng",
                                      "Lat",
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM")

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# My ideal weather criteria includes the following:
    # Max temprature must be greater than 25 but lower than 30 degrees
    # Cloudiness has to be greater than 2
    # Wind speed should be less than 15 m/s

weather_condition_df = city_data_df.loc[(city_data_df["Max Temp"] <30)
                                       & (city_data_df["Max Temp"] >25)
                                       & (city_data_df ["Cloudiness"]> 2)
                                       & (city_data_df["Wind Speed"]<15)]

# Drop any rows with null values
weather_condition_df.dropna()

# Display sample data
weather_condition_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,paracuru,-3.4100,-39.0306,28.39,82,28,5.94,BR,1701832175
15,15,olutanga,7.3106,122.8464,27.93,76,100,2.52,PH,1701832178
31,31,atafu village,-8.5421,-172.5159,29.17,73,100,3.23,TK,1701832061
34,34,kone,-21.0595,164.8658,29.78,53,100,3.45,NC,1701832070
37,37,avarua,-21.2078,-159.7750,27.03,74,75,6.69,CK,1701832182
...,...,...,...,...,...,...,...,...,...,...
529,529,ban ao nang,8.0458,98.8103,27.98,73,93,1.65,TH,1701832322
557,557,campo maior,-4.8278,-42.1686,27.37,70,77,2.71,BR,1701832329
565,565,kapit,2.0167,112.9333,28.91,75,100,1.16,MY,1701832331
567,567,luang prabang,19.8856,102.1347,27.77,65,78,0.48,LA,1701832332


## Step 3: Create a new DataFrame called hotel_df

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(weather_condition_df, columns=["City", "Country","Lat","Lng","Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,paracuru,BR,-3.4100,-39.0306,82,
15,olutanga,PH,7.3106,122.8464,76,
31,atafu village,TK,-8.5421,-172.5159,73,
34,kone,NC,-21.0595,164.8658,53,
37,avarua,CK,-21.2078,-159.7750,74,
...,...,...,...,...,...,...
529,ban ao nang,TH,8.0458,98.8103,73,
557,campo maior,BR,-4.8278,-42.1686,70,
565,kapit,MY,2.0167,112.9333,75,
567,luang prabang,LA,19.8856,102.1347,65,


## Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accomodation.hotel",
          "key":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longi = row ["Lng"]
    lati = row ["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longi},{lati},{radius}"
    params["bias"] = f"proximity::{longi},{lati}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
paracuru - nearest hotel: No hotel found
olutanga - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
pandan - nearest hotel: No hotel found
kavieng - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
sinabang - nearest hotel: No hotel found
yaren - nearest hotel: No hotel found
bamboo flat - nearest hotel: No hotel found
tabou - nearest hotel: No hotel found
alofi - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
taulaga - nearest hotel: No hotel found
kingston - nearest hotel: No hotel found
trincomalee - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
sao miguel - nearest hotel: No hotel found
kismayo - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No

,City,Country,Lat,Lng,Humidity,Hotel Name
1,paracuru,BR,-3.4100,-39.0306,82,No hotel found
15,olutanga,PH,7.3106,122.8464,76,No hotel found
31,atafu village,TK,-8.5421,-172.5159,73,No hotel found
34,kone,NC,-21.0595,164.8658,53,No hotel found
37,avarua,CK,-21.2078,-159.7750,74,No hotel found
...,...,...,...,...,...,...
529,ban ao nang,TH,8.0458,98.8103,73,No hotel found
557,campo maior,BR,-4.8278,-42.1686,70,No hotel found
565,kapit,MY,2.0167,112.9333,75,No hotel found
567,luang prabang,LA,19.8856,102.1347,65,No hotel found


## Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# %%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng",
                                    "Lat",
                                      geo = True,
                                      color = "City",
                                      alpha = 0.8,
                                      size = "Humidity",
                                      tiles = "OSM",
                                  hover_cols = ["Hotel Name","Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)